Resume NER Part 4: Working with Flair NLP

---

In this part we will use flair NLP to train a model on our data and evaluate the results. Please make sure you have set up your Google account and uploaded your files to Google drive. This Notebook should run on Google Colab.

Let's change the working directory to the Google drive where our training data is, and install flair nlp. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir("/content/gdrive/My Drive/SAKI_2019/data")

In [0]:
# download flair library #
! pip install flair

In the next section, we will train a NER model with flair. This code is taken from the flair nlp tutorials section 7. "Training a model" 
https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md



In [0]:
#! mv '/content/gdrive/My Drive/SAKI_2019/data/training_bilou (2).txt' '/content/gdrive/My Drive/SAKI_2019/data/training_bilou.txt'
! ls '/content/gdrive/My Drive/SAKI_2019/data'

 converted_resumes.json		       resources        training_bilou.txt
'Entity Recognition in Resumes.json'   test_bilou.txt


In [0]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

## make sure this describes your file structure
columns = {0: 'text', 1: 'ner'}

# folder where training and test data are
data_folder = '/content/gdrive/My Drive/SAKI_2019/data'

# 1.0 is full data, try a much smaller number like 0.1 to test run the code
downsample = 1.0

## your train file name
train_file = 'training_bilou.txt'

## your test file name
test_file = 'test_bilou.txt'
# 1. get the corpus
corpus: Corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns,
                                                             train_file=train_file,
                                                             test_file=test_file,
                                                           dev_file=None).downsample(downsample)
print(corpus)

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')
print(tag_dictionary.idx2item)


2019-06-18 14:52:31,636 Reading data from /content/gdrive/My Drive/SAKI_2019/data
2019-06-18 14:52:31,640 Train: /content/gdrive/My Drive/SAKI_2019/data/training_bilou.txt
2019-06-18 14:52:31,645 Dev: None
2019-06-18 14:52:31,647 Test: /content/gdrive/My Drive/SAKI_2019/data/test_bilou.txt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:312: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  train_file, column_format
/usr/local/lib/python3.6/dist-packages/flair/data_fetcher.py:318: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  test_file, column_format


Corpus: 9401 train + 1044 dev + 3100 test sentences
[b'<unk>', b'O', b'B-Name', b'L-Name', b'-', b'B-Designation', b'L-Designation', b'B-Degree', b'I-Degree', b'L-Degree', b'I-Designation', b'U-Degree', b'U-Designation', b'I-Name', b'<START>', b'<STOP>']


In [0]:
# 4. initialize embeddings. Experiment with different embedding types to see what gets the best results
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings
from typing import List

embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('glove'),
    # comment in this line to use character embeddings
    CharacterEmbeddings(),

    # comment in these lines to use flair embeddings (needs a LONG time to train :-)
    #FlairEmbeddings('news-forward'),
    #FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='ner',
                                        use_crf=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

## give your model a name and folder of your choice. Your model will be saved there for loading later 
## you can run this notebook many times with different embeddings/params and save the models with different names
model_name = 'resources/taggers/resume-WEglove-CE'

# 7. start training - you can experiment with batch size if you get memory errors
# how many epochs does it take before the model stops showing improvement? Start with a big number like 150, and stop the code cell
# from running at any time - the framework will persist the best model even if you interrupt training. 
trainer.train(model_name,
              learning_rate=0.1,
              mini_batch_size=32,
              #anneal_with_restarts=True,
              max_epochs=20)


2019-06-18 14:53:00,163 ----------------------------------------------------------------------------------------------------
2019-06-18 14:53:00,165 Evaluation method: MICRO_F1_SCORE
2019-06-18 14:53:00,636 ----------------------------------------------------------------------------------------------------
2019-06-18 14:53:02,337 epoch 1 - iter 0/294 - loss 88.17574310
2019-06-18 14:53:23,454 epoch 1 - iter 29/294 - loss 11.84778854
2019-06-18 14:53:44,233 epoch 1 - iter 58/294 - loss 8.29505000
2019-06-18 14:54:08,515 epoch 1 - iter 87/294 - loss 6.89873970
2019-06-18 14:54:32,620 epoch 1 - iter 116/294 - loss 6.12702840
2019-06-18 14:55:02,538 epoch 1 - iter 145/294 - loss 5.47876507
2019-06-18 14:55:23,447 epoch 1 - iter 174/294 - loss 4.97724392
2019-06-18 14:55:45,851 epoch 1 - iter 203/294 - loss 4.69156968
2019-06-18 14:56:07,959 epoch 1 - iter 232/294 - loss 4.47649900
2019-06-18 14:56:30,913 epoch 1 - iter 261/294 - loss 4.25605112
2019-06-18 14:56:53,052 epoch 1 - iter 290/29